In [20]:
# step2_storm_coords_idalia
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
import pickle
from datetime import datetime, timezone
from scipy import stats

from pathlib import Path

import matplotlib.colors
import matplotlib.cm as cm

import xarray as xr
# import xroms
from wavespectra import read_ww3, read_swan, read_ndbc, read_netcdf
from wavespectra.input.swan import read_swans

%run -i storm_coords.py
%run -i wave_stats.py

DATA_DIRECTORY = '/vortexfs1/home/csherwood/proj/NOPP_Idalia/'
DATA_FILENAME = 'hurricane_idalia_drifter_data_v2.pickle'

MODEL_DIRECTORY = '/proj/usgs-share/Projects/Idalia2023/run1/2dspec/'


In [2]:
with open(os.path.join(DATA_DIRECTORY, DATA_FILENAME), 'rb') as handle:
    drifters = pickle.load(handle)

drifter_types = ['spotter', 'dwsd', 'microswift']
pfx = ['S', 'D', 'M']
# `drifters` is a python dictionary keyed by drifter type (spotter,
# dwsd, or microswift)
# `spotter` is a python dictionary of Pandas DataFrames, keyed by
# each drifter ID. The drifter ids can then be accessed as follows:
spotter = drifters['spotter']
spotter_ids = list(spotter.keys())

dwsd = drifters['dwsd']
dwsd_ids = list(dwsd.keys())
    
mswift = drifters['microswift']
mswift_ids = list(mswift.keys())

# # Make nicknames for these drifters to conform with the max. 8 char limit in SWAN filesname
# # after adding three characters for a counter (to differentiate the POINTS).
snames=['S025D','S052D','S055D','S061D','S066D','S095D','S101D','S102D','S103D','S164D',
    'D9690','D1280','D3160', 'D3730','D0060','D0070','D0090','D0250','D8160','D9490','D8010',
    'M0029','M0037','M0046','M0048']

In [3]:
# Do we want time as the index?
#dfo = pd.read_csv('idalia_track_dist_brg.csv', index_col=0)
dfo = pd.read_csv('idalia_track_dist_brg.csv')
dfo

,time_utc,lat,lon,max_wind_speed_ms,radius_to_max_wind_km,distance_to_next_km,bearing_to_next_geo
0,2023-08-24T18:00:00,16.0,-84.0,7.715910,0.00,84.161168,-22.312370
1,2023-08-25T00:00:00,16.7,-84.3,7.715910,0.00,94.253937,-34.255901
2,2023-08-25T06:00:00,17.4,-84.8,7.715910,0.00,85.184904,-38.369813
3,2023-08-25T12:00:00,18.0,-85.3,10.290861,166.68,69.823621,-37.163657
4,2023-08-25T18:00:00,18.5,-85.7,10.290861,166.68,108.557385,-22.735114
5,2023-08-26T00:00:00,19.4,-86.1,10.290861,222.24,100.075434,0.000000
6,2023-08-26T06:00:00,20.3,-86.1,10.290861,222.24,55.597463,0.000000
7,2023-08-26T12:00:00,20.8,-86.1,10.290861,222.24,55.597463,0.000000
8,2023-08-26T18:00:00,21.3,-86.1,12.861341,166.68,23.521217,-118.176226
9,2023-08-27T00:00:00,21.2,-86.3,15.431821,111.12,93.583972,-146.187138


* Get drifter time
* Interpolate to get fractional progress
* Update storm center and orientation
* Interpolate max winds and radius
* Calc. distance and bearing from storm to drifter
* Correct bearing for storm orientation
* Calc. normalized distance
* Calc. xs, ys for drifter location
* Calc. xsn, ysn, for normalized drifter location

In [25]:
def storm_coords( dfo, drifter_time, drifter_lat, drifter_lon ):
    # return tindex, tfrac, latsi, lonsi, brngsi, mx_windi, mx_radi, d2d, brng2d, d2dn, xs, ys, xsn, ysn 

    # fake time to interpolate
    dti = pd.to_datetime( drifter_time )
    # print(dti)
    # find time index in storm track for drifter time
    tindex = np.argwhere( dfo['timestamp'].values <= dti)[-1][0] # final [0] converts result from an array to a single value
    # print(tindex, dfo['timestamp'][tindex])
    # calculate fractional time toward next storm-track location
    tfrac = (dti - dfo['timestamp'][tindex])/(dfo['timestamp'][tindex+1] - dfo['timestamp'][tindex])
    # print(tfrac)
    # print(tindex)
    # print( dfo['lat'][tindex], dfo['lon'][tindex] )
    # print( dfo['lat'][tindex+1], dfo['lon'][tindex+1] )
    # find storm position
    brngsi = dfo['bearing_to_next_geo'][tindex]
    latsi, lonsi = get_point_at_distance(dfo['lat'][tindex],
                                         dfo['lon'][tindex],
                                         tfrac * dfo['distance_to_next_km'][tindex],
                                         brngsi)
    # print(latsi, lonsi)

    # interpolate max wind speed and radius
    mx_windi = dfo['max_wind_speed_ms'][tindex] + tfrac * (dfo['max_wind_speed_ms'][tindex+1] - dfo['max_wind_speed_ms'][tindex])
    mx_radi = dfo['radius_to_max_wind_km'][tindex] + tfrac * (dfo['radius_to_max_wind_km'][tindex+1] - dfo['radius_to_max_wind_km'][tindex]) 
    # print('mx_windi, mx_radi:',mx_windi, mx_radi)

    # calculate distance and bearing from storm to drifter
    d2d, brng2d = dist_bearing( lonsi, latsi, drifter_lon, drifter_lat)
    # print('d2d, brng2d:',d2d, brng2d)

    # rotate into storm coords
    brng2dn = brng2d-dfo['bearing_to_next_geo'][tindex]
    d2dn = d2d/mx_radi

    # convert to xy coordinates
    # not normalized
    xs, ys = xycoord( d2d, brng2dn )

    # normalized
    xsn, ysn, = xycoord( d2dn, brng2dn )
    #print('xs, ys:',xs, ys)
    #print('xsn, ysn:',xsn, ysn)
    return tindex, tfrac, latsi, lonsi, brngsi, mx_windi, mx_radi, d2d, brng2d, d2dn, xs, ys, xsn, ysn

# fake drifter lon, lat
dfo['timestamp']=pd.to_datetime( dfo['time_utc'] )
drifter_time = '2023-08-29T1700'
drifter_lon = -84.6
drifter_lat = 25.6
tindex, tfrac, latsi, lonsi, brngsi, mx_windi, mx_radi, d2d, brng2d, d2dn, xs, ys, xsn, ysn = storm_coords( dfo, drifter_time, drifter_lat, drifter_lon )

2023-08-29 17:00:00
19 2023-08-29 12:00:00
0.8333333333333334
19
23.8 -84.8
25.3 -84.8
25.050000000000004 -84.80000000000001
mx_windi, mx_radi: 40.29693066666667 27.78
d2d, brng2d: 64.37607139778004 18.152455585990513
xs, ys: 20.05614088802889 61.17213239124175
xsn, ysn: 0.7219633149038477 2.202020604436348


In [ ]:
dt = []
time = []
did = []
lat = []
lon = []
hsmod = []
hsobs = []
tpmod = []
tpobs = []
mdirobs  = []
mdirsobs = []
mdirmod  = []
mdirsmod = []
sprd1 = []
sprd1s = []
sigf = []
sigfs = []
stormx_km = []
stormy_km = []
storm_dist_km = []
stormx_n = []
stormy_n = []
storm_dist_n = []
storm_brg = []
storm_mx = []
storm_rad = []



igood=0
icount=0
for dtype in drifter_types :
    drifter_data = drifters[dtype]
    for id in drifter_data.keys():

        # Some rows have times with other data but no wave info (Spotters only)
        only_waves = drifter_data[id]['energy_density'].notnull()

        ipt = 0 # index must stay below 1000 or file names will be too long
        for index, row in drifter_data[id][only_waves].loc['2023-08-29 1200':'2023-08-30 1300'].iterrows():
           
            fn = "{}{:03d}.spc2d".format( snames[icount], ipt)
            pathname = os.path.join(MODEL_DIRECTORY, fn)

            if Path(pathname).is_file():
                # drifter obsservations
                igood+=1
                did.append( snames[icount] )
                dt.append( index )
                time.append( index.strftime('%Y%m%d.%H%m') )
                lat.append( row['latitude'] )
                lon.append( row['longitude'] )
                hsobs.append( row['significant_height'] )
                tpobs.append( row['peak_period'] )
                mdirobs.append( row['mean_direction'] )
                mdirsobs.append( row['mean_directional_spread'] )
                f = row['frequency']
                S = row['energy_density']
                a1 = row['a1']
                b1 = row['b1']
                a2 = row['a2']
                b2 = row['b2']
                # bulk drifter parameters from f, S, a1, b1, etc.
                sigf.append( calc_sigmaf_1d( S, f ) )
                sprd1.append( calc_spread1_a1b1( a1, b1 ) )
                # storm coordinates
                tindex, tfrac, latsi, lonsi, brngsi, mx_windi, mx_radi, d2d, brng2d, d2dn, xs, ys, xsn, ysn = storm_coords( 
                    dfo, index.strftime('%Y-%m-%dT%H%m'), row['latitude'], row['longitude'] )
                stormx_km.append(xs)
                stormy_km.append(ys)
                storm_dist_km.append(d2d)
                stormx_n.append(xsn)
                stormy_n.append(ysn)
                storm_dist_n.append(d2dn)
                storm_brg.append(brngsi)
                storm_mx.append(mx_windi)
                storm_rad.append(mx_radi)
                

                # model output
                df = read_swan( pathname )
                hsmod.append( np.squeeze( df.efth.spec.hs().values ) )
                tpmod.append( np.squeeze( df.efth.spec.tp().values ) )
                mdirmod.append(  np.squeeze( df.efth.spec.dpm().values ) )
                mdirsmod.append( np.squeeze( df.efth.spec.dspr().values ) )
                
                fs = df['freq'].values
                dirs = df['dir'].values
                # # flip the directions, so now directions are where waves come from
                dirs = dirs+180
                dirs[dirs>=360.]=dirs[dirs>=360.]-360.

                directional_bin_width_deg = dirs[2]-dirs[1]
                dirs_r = (np.pi/180.)*dirs
                spec2d = np.squeeze( df.efth.values ) 
                
                # This routine is from Isabel
                # Ss, a1s, a2s, b1s, b2s = to_Fourier( spec2d, fs, dirs_r, directional_bin_width_deg, faxis=0, daxis=1 )
                # Because the conventions for a1, b1 are based on cartesian directions and the SWAN 2dspec is geographic,
                # I think we should switch a1 and b1, and change the sign of a2.
                Ss, b1s, a2s, a1s, b2s = to_Fourier( spec2d, fs, dirs_r, directional_bin_width_deg, faxis=0, daxis=1 )
                a2s = -a2s

                # Almost no energy in high frequencies, so truncate
                igood = np.argwhere(Ss>1.e-8)
                fs = np.squeeze(fs[igood])
                Ss = np.squeeze(Ss[igood])
                a1s = np.squeeze(a1s[igood])
                a2s = np.squeeze(a2s[igood])
                b1s = np.squeeze(b1s[igood])
                b2s = np.squeeze(b2s[igood])
                sigfs.append( calc_sigmaf_1d( Ss, fs ) )
                sprd1s.append( calc_spread1_a1b1( a1s, b1s ) )
               
            ipt += 1           
            
        icount +=1

print(igood)
